In [1]:
# pytorch如何访问参数
import torch
from torch import nn
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))

X = torch.rand(size=(3,4))

net(X)

tensor([[-0.2981],
        [-0.3772],
        [-0.4133]], grad_fn=<AddmmBackward0>)

In [3]:
# 把每一层中的权重打印出来
print(net[2].state_dict())   #显示的是nn.Linear(8,1)

OrderedDict([('weight', tensor([[ 0.2477, -0.2912,  0.2406, -0.1232,  0.0951,  0.1637, -0.1573, -0.2281]])), ('bias', tensor([-0.0934]))])


In [4]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0934], requires_grad=True)
tensor([-0.0934])


In [5]:
net[2].weight.grad == None

True

In [6]:
# 一次性访问所有参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [7]:
# 可以通过名字来打印参数
net.state_dict()['2.bias'].data

tensor([-0.0934])

In [8]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())   # 嵌套四个block1
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.3183],
        [-0.3176],
        [-0.3177]], grad_fn=<AddmmBackward0>)

In [10]:
# 可以通过print来查看网络结构
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [11]:
# 访问第一个主要的块中、第二个子块的第一层的偏置项
rgnet[0][1][0].bias.data

tensor([ 0.2229, -0.4515, -0.3140, -0.4638,  0.4927, -0.0245, -0.1666, -0.2777])

In [12]:
# 如何初始化网络参数
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
        
net.apply(init_normal)  # apply表示对所有net中所有层进行遍历
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0082, -0.0085, -0.0027,  0.0052]), tensor(0.))

In [14]:
# 比如将权重参数初始化为1
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
        
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [15]:
# 利用xavier进行初始化,让训练更稳定
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

# 将参数初始化为42
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

# 对每个层用不同的初始化
net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.1337, -0.3319,  0.4336,  0.3396])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [16]:
# 自定义初始化
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", 
              *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight,-10, 10)  # 均匀初始化
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 9.1502, -6.7448,  8.5009, -6.6052],
        [-0.0000,  7.7387, -0.0000,  0.0000]], grad_fn=<SliceBackward0>)

In [17]:
# 可以直接初始化
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -5.7448,  9.5009, -5.6052])

In [18]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
# 无论怎么更新net网络,两个shared的网络参数应该是相同的
net = nn.Sequential(nn.Linear(4, 8), 
                    nn.ReLU(),
                    shared, 
                    nn.ReLU(),
                    shared, 
                    nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100

# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
